In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle

In [2]:
with open('data.nosync/results.pkl', 'rb') as f:
    results = pickle.load(f)
    
with open('data.nosync/tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)
    
with open('data.nosync/players.pkl', 'rb') as f:
    players = pickle.load(f)

In [3]:
import datetime 
import re

def get_year(tournament):
    parsed_date_str = re.search(r'^[^T]*', tournament['dateStart']).group(0)
    return datetime.date.fromisoformat(parsed_date_str).year
    

# filter tournaments only for 2019 and 2020 years
def filter_tournaments(tournaments, years_range):
    filtered_tournaments = {}
    for idx in tournaments:
        if years_range[0] <= get_year(tournaments[idx]) <= years_range[1]:
            filtered_tournaments[idx] = tournaments[idx]
    return filtered_tournaments

In [4]:
tournaments = filter_tournaments(tournaments, (2019, 2020))

In [5]:
def filter_results(results, filtered_tournaments):
    filtered = {}
    for idx in results:
        if idx in filtered_tournaments:
            teams = results[idx]
            for team in teams:
                if 'mask' in team and team['mask']:
                    team['mask'] = re.sub(r'X|\?', '0', team['mask'])
                    filtered.setdefault(idx, []).append(team)
    return filtered

In [6]:
results = filter_results(results, tournaments)

In [7]:
train_tours_ids = [t for t in results if get_year(tournaments[t]) == 2019]
test_tours_ids = [t for t in results if get_year(tournaments[t]) == 2020]

In [8]:
assert len(train_tours_ids) + len(test_tours_ids) == len(results)

### Подготавливаем датасет для тренировки

In [9]:
def construct_dataset(tour_ids, results, tournaments):
    dataset = []
    base_qid = 0
    for tour_id in tour_ids:
        at_least_one = False
        total_questions = sum(tournaments[tour_id]['questionQty'].values())
        for team in results[tour_id]:
            if len(team['mask']) == total_questions:
                team_id = team['team']['id']
                at_least_one = True
                for i, res in enumerate(team['mask']):
                    for member in team['teamMembers']:
                        player_id = member['player']['id']
                        dataset.append([team_id, player_id, base_qid + i, int(res)])        
        if at_least_one:
            base_qid += total_questions
                    
    return pd.DataFrame(columns=['Team ID', 'Player ID', 'Question ID', 'isCorrect'], data=dataset)

In [10]:
train_dataset = construct_dataset(train_tours_ids, results, tournaments)
test_dataset = construct_dataset(test_tours_ids, results, tournaments)

Кодируем всех игроков и вопросы в one-hot векторы. Вес у соответствующего элемента one-hot вектора игрока будет значить его силу, а вес при соответствующем элемент вектора вопросов значит его "легкость"

In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

OHE = OneHotEncoder(handle_unknown='ignore')
X_train_oh = OHE.fit_transform(train_dataset[['Player ID', 'Question ID']])
X_test_oh = OHE.transform(test_dataset[['Player ID', 'Question ID']])
y_train = train_dataset['isCorrect']
y_test = test_dataset['isCorrect']

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(penalty='l2', solver='liblinear').fit(X_train_oh, y_train)

In [13]:
num_uniq_cols = train_dataset.nunique()
num_players = num_uniq_cols[1]
num_questions = num_uniq_cols[2]
player_weights = logreg.coef_[0, :num_players]
question_wights = logreg.coef_[0, -num_questions:]

In [14]:
def fullname_from_coef(coef_idx, encoder, players):
    player_id = encoder.categories_[0][coef_idx]
    fullname = players[player_id]["name"] + ' '
    fullname += (players[player_id]["patronymic"] + ' ') if players[player_id]["patronymic"] else ""
    fullname += players[player_id]["surname"]
    return fullname

### 20 лучший игроков (с наибольшим весом)

In [15]:
top_20_coefs = np.argsort(player_weights)[-20:][::-1]
for i, coef_idx in enumerate(top_20_coefs):
    fullname = fullname_from_coef(coef_idx, OHE, players)
    tabulation = "\t\t" if len(fullname) < 29 else "\t"
    print(f'{i + 1}. {fullname}' +
          tabulation +
          f'вес: {player_weights[coef_idx]:.3f}')

1. Максим Михайлович Руссо		вес: 4.160
2. Александра Владимировна Брутер	вес: 4.032
3. Иван Николаевич Семушин		вес: 3.987
4. Михаил Владимирович Савченков	вес: 3.898
5. Сергей Леонидович Спешков		вес: 3.820
6. Артём Сергеевич Сорожкин		вес: 3.819
7. Станислав Григорьевич Мереминский	вес: 3.702
8. Михаил Ильич Левандовский		вес: 3.645
9. Ирина Сергеевна Прокофьева		вес: 3.601
10. Сергей Игоревич Николенко		вес: 3.585
11. Илья Сергеевич Новиков		вес: 3.555
12. Антон Владимирович Саксонов		вес: 3.554
13. Александр Витальевич Либер		вес: 3.537
14. Игорь Викторович Мокин		вес: 3.516
15. Александр Владимирович Мосягин	вес: 3.514
16. Алексей Владимирович Гилёв		вес: 3.510
17. Михаил Сергеевич Царёв		вес: 3.503
18. Дмитрий Александрович Карякин	вес: 3.497
19. Александр Валерьевич Марков		вес: 3.495
20. Наталья Евгеньевна Горелова		вес: 3.488


### Собираем предсказания по турнирам

Вес команды - это среднее весов ее участников. Чем больше вес, тем лучше команда

In [16]:
def gen_pid_weight_dict(model, unique_pids, model_idx):
    pid_to_weight = {}
    for i, pid in enumerate(unique_pids):
        if len(model.coef_.shape) == 2:
            pid_to_weight[pid] = model.coef_[0][model_idx[i]]
        elif len(model.coef_.shape) == 1:
            pid_to_weight[pid] = model.coef_[model_idx[i]]
    return  pid_to_weight 

In [17]:
def generate_predictions(test_tours_ids, results, pid_to_weight):
    pred_results = {}
    for tour_id in test_tours_ids:
        if tour_id in results:
            pred_results[tour_id] = []
            for team in results[tour_id]:
                team_weight = 0
                n = 0
                for member in team['teamMembers']:
                    team_weight += pid_to_weight.get(member['player']['id'], 0)
                    n += 1
                if n != 0:
                    team_weight = team_weight / n
                pred_results[tour_id].append([team['team']['id'], team_weight])
            pred_results[tour_id] = np.array(pred_results[tour_id], dtype=int)
    return pred_results

In [18]:
def generate_true_results(test_tours_ids, results):
    true_results = {}
    for tour_id in test_tours_ids:
        if tour_id in results:
            true_results[tour_id] = []
            for team in results[tour_id]:
                true_results[tour_id].append([team['team']['id'], team['questionsTotal']])
            true_results[tour_id] = np.array(true_results[tour_id], dtype=int)
    return true_results

In [19]:
from scipy.stats import spearmanr, kendalltau
import math

def calculate_correlations(pred_results, true_results):
    scorrs = []
    kcorrs = []
    for tour_id in true_results:
        if tour_id in results:
            scorr = spearmanr(true_results[tour_id][:, 1],
                                         pred_results[tour_id][:, 1])[0]
            if not math.isnan(scorr):
                scorrs.append(scorr)

            kcorr = kendalltau(true_results[tour_id][:, 1],
                                         pred_results[tour_id][:, 1])[0]
            if not math.isnan(kcorr):
                kcorrs.append(kcorr)
    
    return np.mean(scorrs), np.mean(kcorrs)

In [20]:
unique_pids = train_dataset['Player ID'].unique()
model_idx = OHE.transform(np.concatenate((unique_pids[:, None],
                                              -1*np.ones(num_players, dtype=int)[:, None]), axis=1)).nonzero()[1]
    
pid2weight = gen_pid_weight_dict(logreg, unique_pids, model_idx)
pred_results = generate_predictions(test_tours_ids, results, pid2weight)
true_results = generate_true_results(test_tours_ids, results)

corrs = calculate_correlations(pred_results, true_results)
print(f"Mean Spearman's correlation:{corrs[0]}")
print(f"Mean Kendall's correlation: {corrs[1]}")

Mean Spearman's correlation:0.6946516860136752
Mean Kendall's correlation: 0.5936957293306635


/Users/ilya/.local/share/virtualenvs/ML2-QrfXVegw-python/lib/python3.8/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


###  Теперь применим EM алгоритм
Скрытая переменная $z_{ij}$ - ответил ли $i$ участник на $j$ вопрос.
Для EM-алгоритма нам нужно мат ожидание $E(z_{ij})$.
Если команда $i$-го участника не ответила на $j$ вопрос, то будем считать $z_{ij} = 0$ и $E(z_{ij}) = 0$.
Если команда ответила верно, то будем считать, что $E(z_{ij})$ - вероятность того что $z_{ij} = 1$, при условии, что кто-то из команды все-таки ответил на вопрос. Данную условную вероятность (или же просто  $E(z_{ij})$) будем моделировать с помощью линейной регрессии, которая будет учиться предсказывать $inverse\_sigmoid(E(z_{ij}))$. Тогда $\sigma(linreg\_output)$ ~ $E(z_{ij})$.
Итак, 

E-шаг: вычислим $E(z_{ij})$, через с помощью предсказаний линейной регрессии, основанной на зафиксированных параметрах "скилов" игроков и "легкости" вопросов.

М-шаг: обучим линейную регресиию предсказывать только что посчитанные $E(z_{ij})$.

In [21]:
from sklearn.linear_model import Ridge, LinearRegression

In [22]:
def E_step(X_train, preds):
    tmp_df = X_train.copy()
    tmp_df['ones_minus_probs'] = 1 - preds
    tmp_df['group_product'] = tmp_df.groupby(['Team ID', 'Question ID'])['ones_minus_probs'].transform('prod')
    tmp_df['E_z'] = (preds / (1 - tmp_df['group_product'])) * tmp_df['isCorrect']
    return tmp_df['E_z'].values

In [23]:
def M_step(X_train_onehot, z_train):
    linreg = LinearRegression().fit(X_train_onehot, z_train)
    return linreg

In [24]:
def sig(x):
    return 1 / (1 + np.exp(-x))

def inv_sig(y):
    y = np.clip(y, 1e-8, 1 - 1e-8)
    return np.log(y / (1 - y))

In [25]:
num_iter = 5

logreg = LogisticRegression(penalty='l2', solver='liblinear').fit(X_train_oh, y_train)
train_preds = logreg.predict_proba(X_train_oh)[:, 1]

pid2weight = gen_pid_weight_dict(logreg, unique_pids, model_idx)
pred_results = generate_predictions(test_tours_ids, results, pid2weight)
corrs = calculate_correlations(pred_results, true_results)
print(f"Baseline:")
print(f"Mean Spearman's correlation:{corrs[0]}")
print(f"Mean Kendall's correlation: {corrs[1]}")
print()
    
best_model = None
max_scorr = 0

for i in range(num_iter):
    print(f'Iteration {i} is started...')
    # E-step
    E_z = E_step(train_dataset, train_preds)
    # M-step
    linreg = M_step(X_train_oh, inv_sig(E_z))
    
    train_preds = sig(linreg.predict(X_train_oh))
    
    
    pid2weight = gen_pid_weight_dict(linreg, unique_pids, model_idx)
    pred_results = generate_predictions(test_tours_ids, results, pid2weight)
    corrs = calculate_correlations(pred_results, true_results)
    print(f"Mean Spearman's correlation:{corrs[0]}")
    print(f"Mean Kendall's correlation: {corrs[1]}")
    print()
    
    if corrs[0] > max_scorr:
        max_scorr = corrs[0]
        best_model = logreg

/Users/ilya/.local/share/virtualenvs/ML2-QrfXVegw-python/lib/python3.8/site-packages/scipy/stats/stats.py:4264: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


Baseline:
Mean Spearman's correlation:0.6946516860136752
Mean Kendall's correlation: 0.5936957293306635

Iteration 0 is started...
Mean Spearman's correlation:0.7625189306901213
Mean Kendall's correlation: 0.6335766525433221

Iteration 1 is started...
Mean Spearman's correlation:0.7615111031729237
Mean Kendall's correlation: 0.6339508260350866

Iteration 2 is started...
Mean Spearman's correlation:0.7656064760761063
Mean Kendall's correlation: 0.6389242840201143

Iteration 3 is started...
Mean Spearman's correlation:0.7646738143060503
Mean Kendall's correlation: 0.6372318701286596

Iteration 4 is started...
Mean Spearman's correlation:0.7653152723335793
Mean Kendall's correlation: 0.6383696819959492



Обучил только 5 итераций только при последнем запуске.
Запустал 50 итераций, но последующие итерации не улучшают заметно качество модели. Оно всегда колеблется в районе 0.75-0.765 по коэффициенту корреляции Спирмана

In [26]:
def generate_tour_rating(qid_to_weight, tour_ids, tournaments):
    tour_rating = []
    base_qid = 0
    for tour_id in tour_ids:
        weight = 0
        n = sum(tournaments[tour_id]['questionQty'].values())
        for i in range(n):
            weight += qid_to_weight[base_qid + i]
        weight /= n
        tour_rating.append((tour_id, weight))
        base_qid += n
    return sorted(tour_rating, key=lambda x: x[1])

In [27]:
unique_qids = train_dataset['Question ID'].unique()
model_idx = OHE.transform(np.concatenate((-1*np.ones(num_questions, dtype=int)[:, None],
                                         unique_qids[:, None]), axis=1)).nonzero()[1]

qid2weight = gen_pid_weight_dict(linreg, unique_qids, model_idx)
tour_rating = generate_tour_rating(qid2weight, train_tours_ids, tournaments)

#### Топ самых сложных турниров.
Сложность турнира считается как средняя сложность всех его вопросов. Так как коэффициенты модели показывают "легкость" вопросов, то самый сложный турнир будет иметь самый низкий показатель веса, усредненного по всем его вопросам.

In [28]:
top_complex_tournaments = tour_rating[:20]
for i, (tour_id, _) in enumerate(top_complex_tournaments):
    name = tournaments[tour_id]['name']
    print(f'{i + 1}. {name}')

1. Чемпионат Санкт-Петербурга. Первая лига
2. Угрюмый Ёрш
3. Синхрон высшей лиги Москвы
4. Воображаемый музей
5. Первенство правого полушария
6. Чемпионат Мира. Этап 2. Группа В
7. Чемпионат России
8. Знание – Сила VI
9. Чемпионат Мира. Этап 2. Группа А
10. Ускользающая сова
11. Записки охотника
12. Зеркало мемориала памяти Михаила Басса
13. Чемпионат Минска. Лига А. Тур четвёртый
14. Чемпионат Мира. Этап 3. Группа В
15. Чемпионат Мира. Этап 2 Группа С
16. Чемпионат Мира. Финал. Группа С
17. Львов зимой. Адвокат
18. Чемпионат Мира. Этап 1. Группа С
19. Чемпионат Мира. Этап 3. Группа С
20. Мемориал Дмитрия Коноваленко


#### Топ самых легкий турниров

In [29]:
top_easy_tournaments = tour_rating[-20:][::-1]
for i, (tour_id, _) in enumerate(top_easy_tournaments):
    name = tournaments[tour_id]['name']
    print(f'{i + 1}. {name}')

1. One ring - async
2. (а)Синхрон-lite. Лига старта. Эпизод V
3. Синхрон Лиги Разума
4. Школьная лига. I тур.
5. Школьная лига. III тур.
6. Студенческий чемпионат Калининградской области
7. Школьная лига
8. Школьный Синхрон-lite. Выпуск 2.5
9. (а)Синхрон-lite. Лига старта. Эпизод VII
10. Школьная лига. II тур.
11. (а)Синхрон-lite. Лига старта. Эпизод IX
12. (а)Синхрон-lite. Лига старта. Эпизод III
13. Межфакультетский кубок МГУ. Отбор №4
14. Школьный Синхрон-lite. Выпуск 3.1
15. Школьный Синхрон-lite. Выпуск 3.3
16. (а)Синхрон-lite. Лига старта. Эпизод IV
17. Малый кубок Физтеха
18. (а)Синхрон-lite. Лига старта. Эпизод X
19. Школьный Синхрон-lite. Выпуск 2.3
20. Второй тематический турнир имени Джоуи Триббиани


Как мы можем видеть, в топе самых сложных турниров находятся чемпионаты мира, а в топе самых легких школьные турниры.